Import necessary packages and creating function std_name to standardized table and column name as per requirement

In [1]:
import requests,zipfile,io,sys,os,os.path,shutil,csv
from os.path import join
from __future__ import unicode_literals
def std_name(name,kind = "t"):
    name.strip()
    name=name.replace(" ","_")
    name=name.replace("\t","_")
    name=name.replace("%","pct")
    name=name.replace("/","_")
    name=name.replace("-","_")
    name=name.replace("#","_num")
    name=name.replace(",","_")
    name=name.lower()
    if kind == 't':
        if not name[0].isalpha():
            name="t_"+name
    if kind == 'c':
        if not name[0].isalpha():
            name="c_"+name
    return name

def str_convertor(lst): 
    return[str(elem) for elem in lst]

Downloading Zip file

In [2]:
results = requests.get('https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1?content_type=application%2Fzip%3%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip')
data_zip = zipfile.ZipFile(io.BytesIO(results.content))

Creating staging directory

In [3]:
wd = os.getcwd()
staging_path = wd + "\\staging"
if os.path.exists(staging_path):
    shutil.rmtree(staging_path)

Extracting Zip file in staging directory

In [4]:
data_zip.extractall("staging")

Creating new files with utf-8 encoding and tranforming file name and headers as per requirement and importing data from old files to new files

In [5]:
os.chdir(staging_path)
files = os.listdir()
for file in files:
    if not file.endswith(".csv"):
        os.remove(file)
        continue
    original_file=file + "-original"
    os.rename(file,original_file)
    new_name=std_name(file,"t")
    with io.open(original_file, 'r', encoding="windows-1252") as f_in:
        f_in = csv.reader(x.replace('\0', '') for x in f_in)
        with io.open(new_name, 'w', encoding="utf8", newline='') as f_o:
            f_out = csv.writer(f_o,delimiter=',')
            for i, line in enumerate(f_in):
                if line == [] or line == [' ']:
                    continue
                elif i == 0:
                    s = []
                    for element in range(0,len(line)):
                        s.append(std_name(line[element],"c"))
                    f_out.writerow(str_convertor(s))
                else:
                    f_out.writerow(str_convertor(line))
os.chdir(wd)

Importing sqlite package

In [6]:
import sqlite3

Creating new database medicare_hospital_compare.db

In [7]:
db = sqlite3.connect('medicare_hospital_compare.db')
c = db.cursor()

Creating tables for each files and importing data fron csv files to database tables

In [8]:
os.chdir(staging_path)
tables = os.listdir()
for table in tables:
    if table.endswith(".csv-original"):
        continue
    else:
        sql_table = table.rstrip('.csv')
        with io.open(table, 'r', encoding="utf8") as t_in:
            table_in = csv.reader(t_in)
            h = next(table_in)
            header = []
            for i in range(0,len(h)):
                header.append(h[i] + ' ' + 'TEXT')
                str1 = ','.join(header)
            create_str ='CREATE TABLE IF NOT EXISTS ' + sql_table + ' ' +'(' + str1 + ')'
            c.execute(create_str)
            stmt = "INSERT INTO " + sql_table + '({0})' + ' VALUES({1});'
            query = stmt.format(','.join(h), ','.join('?' * len(h)))
            for line in table_in:
                c.execute(query, line)
os.chdir(wd)
db.commit()
db.close()

Downloading excel file from file

In [9]:
dls = "http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx"
resp = requests.get(dls)
with io.open('ranking.xlsx', 'wb') as output:
    output.write(resp.content)

Converting excel file to pandas dataframe

In [10]:
import pandas as pd
xl = pd.ExcelFile("ranking.xlsx")

Tranforming column names and creating sqlite table for each sheets

In [11]:
db = sqlite3.connect('medicare_hospital_compare.db')
c = db.cursor()
sheets = xl.sheet_names
for sheet in sheets:
    df = pd.DataFrame()
    header = []
    n_t = std_name(sheet)
    df = pd.read_excel(xl,sheetname=sheet, dtype=str) 
    h = list(df.columns.values)
    for value in h:
        header.append(std_name(value,"c"))
    df.columns = header
    df.applymap(str)
    df.to_sql(n_t, db, if_exists="replace",index=False)
db.commit()
db.close()

Merging tables in database and creating new merged tables according to output requirements and exporting merged tables as a pandas dataframes

In [12]:
db = sqlite3.connect('medicare_hospital_compare.db')
c = db.cursor()
query1 = 'DROP TABLE IF EXISTS merge_table'
query2 = 'SELECT * from hospital_national_ranking ORDER BY provider_id'
query3 = 'SELECT * from hospital_general_information ORDER BY provider_id'
query4 = 'CREATE TABLE IF NOT EXISTS merge_table AS SELECT b.ranking, a.provider_id, a.hospital_name, a.city, a.state, a.county_name FROM hospital_general_information AS a JOIN hospital_national_ranking AS b ON a.provider_id = b.provider_id'
query5 = 'SELECT * from merge_table ORDER BY state'
query6 = 'SELECT * from focus_states ORDER BY state_abbreviation'
query7 = 'DROP TABLE IF EXISTS final_table'
query8 = 'CREATE TABLE IF NOT EXISTs final_table AS SELECT merge_table.*, focus_states.state_name FROM focus_states JOIN merge_table ON merge_table.state = focus_states.state_abbreviation'
c.execute(query1)
c.execute(query2)
c.execute(query3)
c.execute(query4)
c.execute(query5)
c.execute(query6)
c.execute(query7)
c.execute(query8)
db.commit
master_table = pd.read_sql_query("select * from final_table;", db)
national_table = pd.read_sql_query("select * from merge_table;", db)
measure_table = pd.read_sql_query("select state, measure_id, measure_name, score from timely_and_effective_care___hospital", db)
db.close()

Changing the datatypes to perform analytics and creating list and dictionary to use for loops

In [13]:
states= list(master_table.state_name.unique())
abb= list(master_table.state.unique())
state_dict = dict(zip(abb, states))
master_table[['ranking']] = master_table[['ranking']].astype(int)
national_table[['ranking']] = national_table[['ranking']].astype(int)
title = ['Provider ID','Hospital Name','City','State','County']

Creating output excel file

In [14]:
from pandas import ExcelWriter
writer = pd.ExcelWriter('hospital_ranking.xlsx')

Writing first sheet according to requrements of output 

In [15]:
Nationwide = national_table.sort_values('ranking')
Nationwide = Nationwide.head(n=100)
del Nationwide['ranking']
Nationwide.columns = title
Nationwide.to_excel(writer,'Nationwide',index=False)

Writing rest of the sheets according to requrements of output 

In [16]:
d = {}
for state in states:
    temp = master_table[master_table['state_name'] == state]
    d[state] = temp.sort_values('ranking')
    d[state] = d[state].head(n=100)
    del d[state]['state_name']
    del d[state]['ranking']
    d[state].columns = title
    d[state].to_excel(writer,state,index=False)
writer.save()
writer.close()

Tranforming pandas dataframe for second output file

In [17]:
measures = measure_table[measure_table['score'].astype(str).str.isdigit()]
measures['score'] = measures['score'].apply(pd.to_numeric, errors='coerce')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Writing first sheet according to requrements of output 

In [18]:
writer2 = pd.ExcelWriter('measure_statistics.xlsx')
nation_stat =  measures.groupby(['measure_id','measure_name']).agg(['min','max','mean','std'])
nation_stat=nation_stat.reset_index()
col_name = ['Measure ID','Measure Name','Minimum','Maximum','Average','Standard Deviation']
nation_stat.columns = col_name
nation_stat.to_excel(writer2,'Nationwide',index=False)

Creating new tranformed dataframe for rest of the sheets

In [19]:
grouped_data = measures.groupby(['state', 'measure_id','measure_name']).agg(['min','max','mean','std'])
grouped_data=grouped_data.reset_index()
col_name = ['state', 'Measure ID','Measure Name','Minimum','Maximum','Average','Standard Deviation']
grouped_data.columns = col_name
grouped_data['Standard Deviation'].fillna(0, inplace=True)

Writing rest of the sheets according to requrements of output 

In [20]:
for a,f in state_dict.items():
    temp = grouped_data[grouped_data['state'] == a]
    del temp['state']
    temp.to_excel(writer2,f,index=False)
writer2.save()
writer2.close()